In [33]:
import cv2
import numpy as np


In [34]:
orig_lena = cv2.imread('lena.bmp', 0)

# Spec

Write a program which does:

(a) Generate noisy images with gaussian noise(amplitude of 10 and 30)  
(b) Generate noisy images with salt-and-pepper noise( probability 0.1 and 0.05)  
(c) Use the 3x3, 5x5 box filter on images generated by (a)(b)  
(d) Use 3x3, 5x5 median filter on images generated by (a)(b)  
(e) Use both opening-then-closing and closing-then opening filter (using the octogonal 3-5-5-5-3 kernel, value = 0) on images generated by (a)(b)  

You must calculate the signal-to-ratio (SNR) for each instance(4 noisy images and 24 processed images)  


# Generate noisy images

## (a) gaussian noise
(amplitude of 10 and 30)
> from ppt p. 70

We generate the additive white Gaussian noise by the formula:  

$$
I(nim, i, j) = I(im, i, j) + amplitude＊N(0, 1)
$$  

Where $nim$ represents the resulting noisy image, and $im$ represents the original image.  
We calculate the intensity value of the $(i,j)$ pixel in the noisy image by using the original intensity value $I(im, i, j)$ plus $amplitude＊N(0, 1)$.

In [35]:
def add_gaussian_noise(orig_image, mu, sigma, amplitude):
    return orig_image + amplitude * np.random.normal(mu, sigma, orig_image.shape)

In [54]:
Gaussian_10 = add_gaussian_noise(orig_lena, 0, 1, 10)
Gaussian_30 = add_gaussian_noise(orig_lena, 0, 1, 30)

cv2.imwrite('prob_a_result/Gaussian_10.bmp', Gaussian_10)
cv2.imwrite('prob_a_result/Gaussian_30.bmp', Gaussian_30)


True

# (b) salt-and-pepper noise
> from ppt p.70
(probability 0.1 and 0.05)

## Explanation aobut the salt-and-pepper noise

We generate the salt-and-pepper noise by the formula:  

$$
I(nim, i, j) = 
\begin{cases} 
0, & \text{if } uniform(0, 1) < threshold \\
255, & \text{if } uniform(0, 1) > 1 - threshold \\
I(im, i, j), & \text{otherwise}
\end{cases}
$$

The meaning behind this formula is that we randomly generate black (pepper) and white (salt) dots scattered across an image, in order to simulates errors that occur in image transmission or sensor faults, where some pixels are completely replaced by extreme values.

- Salt (white): Pixel values are randomly set to the maximum intensity (255).
- Pepper (black): Pixel values are randomly set to the minimum intensity (0).

## Explanation about the code

To generate the salt-and-pepper noise, we:

1. generate a random number uniformly distributed between 0 and 1 for each pixel
2. compare it with the threshold
> If it is less than the threshold, we set the pixel value to 0 (pepper noise);  
> if it is greater than 1 - threshold, we set the pixel value to 255 (salt noise);  
> otherwise, we keep the original pixel value.

Note: The threshold determines the signal-to-noise ratio (SNR), which we should try 0.1 and 0.05.
> The meaning of threshold is that this value controls the amount of noise, the higher the threshold, the more noisy pixels (more black and white dots). 

### Some implementation details

1. We use `np.random.rand()` to generate a random number uniformly distributed between 0 and 1 for each pixel.

We use `np.random.rand()` instead of `np.random.uniform()` because the former is faster, and we don't need to specify the range.  
(while we can specify `low`, `high` in `np.random.uniform()`)

```python
random.rand(d0, d1, ..., dn)
```
> The parameters `d0, d1, …, dn` are the dimensions of the returned array. (The return is a ndarray of shape (d0, d1, …, dn))  

$\rightarrow$ generates an array of the given shape and populate it with random samples from a uniform distribution over [0, 1).

2. We use `*` to unpack the shape of the original image.
3. Both `pepper_mask` and `salt_mask` are boolean ndarrays.


In [37]:
def add_salt_and_pepper_noise(orig_image, threshold):
    noisy_image = orig_image.copy()
    pepper_mask = np.random.rand(*orig_image.shape) < threshold
    salt_mask = np.random.rand(*orig_image.shape) > 1 - threshold
    noisy_image[pepper_mask] = 0
    noisy_image[salt_mask] = 255
    return noisy_image


In [38]:
salt_n_pepper_01 = add_salt_and_pepper_noise(orig_lena, 0.1)
salt_n_pepper_005 = add_salt_and_pepper_noise(orig_lena, 0.05)

cv2.imwrite('prob_b_result/salt_n_pepper_01.bmp', salt_n_pepper_01)
cv2.imwrite('prob_b_result/salt_n_pepper_005.bmp', salt_n_pepper_005)


True

# (c) Use the 3x3, 5x5 box filter 
> from ppt p.15-17  
> textbook p.304

on images generated by (a)(b)

## Explanation about the box filter

The box filter is an operator that computes the equally weighted average.
The box operator with a $(2M + 1) \times (2N + 1)$ neighborhood smoothing image $f$ is defined by:

$$
k(r,c) = \frac{1}{(2M + 1)(2N + 1)} \sum_{r'=-M}^{M} \sum_{c'=-N}^{N} f(r+r', c+c')
$$
> For example, if we use a 3x3 box filter, we sum over the 3x3 neighborhood centered on $(r,c)$ and divide by $9$.


## Explanation about the code

We implement this function by the following steps:
1. calculate $M, N$ as in the formula
2. add padding to the original image
> We add border width $M$ to the top and bottom directions, and border width $N$ to the left and right directions.
3. calculate the average of the neighborhood and assign it to the corresponding pixel in the output image

Note: I use `cv2.copyMakeBorder()` to add padding to the original image.

```python
cv2.copyMakeBorder(src, top, bottom, left, right, borderType, value=None)
```
> For the kind of border, I use `cv2.BORDER_REFLECT`




In [39]:
def box_filter(orig_image, filter_size_row, filter_size_col):
    M = (filter_size_row - 1) // 2
    N = (filter_size_col - 1) // 2

    img_with_padding = cv2.copyMakeBorder(orig_image, M, M, N, N, cv2.BORDER_REFLECT)
    box_filtered_image = np.zeros_like(orig_image)
    for i in range(box_filtered_image.shape[0]):
        for j in range(box_filtered_image.shape[1]):
            box_filtered_image[i, j] = np.mean(img_with_padding[i:i + filter_size_row, j:j + filter_size_col])
    return box_filtered_image


In [40]:

# aim: generate the box filtered images
# subaim: Using the 3x3 box filter
box_filtered_3x3_amp10 = box_filter(Gaussian_10, 3, 3)
box_filtered_3x3_amp30 = box_filter(Gaussian_30, 3, 3)
box_filtered_3x3_salt_n_pepper_01 = box_filter(salt_n_pepper_01, 3, 3)
box_filtered_3x3_salt_n_pepper_005 = box_filter(salt_n_pepper_005, 3, 3)

# subaim: Using the 5x5 box filter
box_filtered_5x5_amp10 = box_filter(Gaussian_10, 5, 5)
box_filtered_5x5_amp30 = box_filter(Gaussian_30, 5, 5)
box_filtered_5x5_salt_n_pepper_01 = box_filter(salt_n_pepper_01, 5, 5)
box_filtered_5x5_salt_n_pepper_005 = box_filter(salt_n_pepper_005, 5, 5)

# aim: save the box filtered images
# subaim: save the 3x3 box filtered images
cv2.imwrite('prob_c_result/box_filtered_3x3_amp10.bmp', box_filtered_3x3_amp10)
cv2.imwrite('prob_c_result/box_filtered_3x3_amp30.bmp', box_filtered_3x3_amp30)
cv2.imwrite('prob_c_result/box_filtered_3x3_salt_n_pepper_01.bmp', box_filtered_3x3_salt_n_pepper_01)
cv2.imwrite('prob_c_result/box_filtered_3x3_salt_n_pepper_005.bmp', box_filtered_3x3_salt_n_pepper_005)

# subaim: save the 5x5 box filtered images
cv2.imwrite('prob_c_result/box_filtered_5x5_amp10.bmp', box_filtered_5x5_amp10)
cv2.imwrite('prob_c_result/box_filtered_5x5_amp30.bmp', box_filtered_5x5_amp30)
cv2.imwrite('prob_c_result/box_filtered_5x5_salt_n_pepper_01.bmp', box_filtered_5x5_salt_n_pepper_01)
cv2.imwrite('prob_c_result/box_filtered_5x5_salt_n_pepper_005.bmp', box_filtered_5x5_salt_n_pepper_005)


True

# (d) Use 3x3, 5x5 median filter

on images generated by (a)(b)

The median filter function is implemented similarly to the box filter, but instead of calculating the average, we calculate the median using `np.median()`.

In [41]:
def median_filter(orig_image, filter_size_row, filter_size_col):
    M = (filter_size_row - 1) // 2
    N = (filter_size_col - 1) // 2
    img_with_padding = cv2.copyMakeBorder(orig_image, M, M, N, N, cv2.BORDER_REFLECT)
    median_filtered_image = np.zeros_like(orig_image)
    for i in range(median_filtered_image.shape[0]):
        for j in range(median_filtered_image.shape[1]):
            median_filtered_image[i, j] = np.median(img_with_padding[i:i + filter_size_row, j:j + filter_size_col])
    return median_filtered_image


In [42]:
# aim: generate the median filtered images
# subaim: Using the 3x3 median filter
median_filtered_3x3_amp10 = median_filter(Gaussian_10, 3, 3)
median_filtered_3x3_amp30 = median_filter(Gaussian_30, 3, 3)
median_filtered_3x3_salt_n_pepper_01 = median_filter(salt_n_pepper_01, 3, 3)
median_filtered_3x3_salt_n_pepper_005 = median_filter(salt_n_pepper_005, 3, 3)

# subaim: Using the 5x5 median filter
median_filtered_5x5_amp10 = median_filter(Gaussian_10, 5, 5)
median_filtered_5x5_amp30 = median_filter(Gaussian_30, 5, 5)
median_filtered_5x5_salt_n_pepper_01 = median_filter(salt_n_pepper_01, 5, 5)
median_filtered_5x5_salt_n_pepper_005 = median_filter(salt_n_pepper_005, 5, 5)

# aim: save the median filtered images
# subaim: save the 3x3 median filtered images
cv2.imwrite('prob_d_result/median_filtered_3x3_amp10.bmp', median_filtered_3x3_amp10)
cv2.imwrite('prob_d_result/median_filtered_3x3_amp30.bmp', median_filtered_3x3_amp30)
cv2.imwrite('prob_d_result/median_filtered_3x3_salt_n_pepper_01.bmp', median_filtered_3x3_salt_n_pepper_01)
cv2.imwrite('prob_d_result/median_filtered_3x3_salt_n_pepper_005.bmp', median_filtered_3x3_salt_n_pepper_005)

# subaim: save the 5x5 median filtered images
cv2.imwrite('prob_d_result/median_filtered_5x5_amp10.bmp', median_filtered_5x5_amp10)
cv2.imwrite('prob_d_result/median_filtered_5x5_amp30.bmp', median_filtered_5x5_amp30)
cv2.imwrite('prob_d_result/median_filtered_5x5_salt_n_pepper_01.bmp', median_filtered_5x5_salt_n_pepper_01)
cv2.imwrite('prob_d_result/median_filtered_5x5_salt_n_pepper_005.bmp', median_filtered_5x5_salt_n_pepper_005)


True

# (e) Use both opening-then-closing and closing-then-opening filter 

(using the octogonal 3-5-5-5-3 kernel, value = 0) on images generated by (a)(b)

For this part, we use the functions implemented in homework 5.

In [43]:
kernel = [[-2, -1], [-2, 0], [-2, 1],                         # 3
            [-1, -2], [-1, -1], [-1, 0], [-1, 1], [-1, 2],    # 5
            [0, -2], [0, -1], [0, 0], [0, 1], [0, 2],         # 5
            [1, -2], [1, -1], [1, 0], [1, 1], [1, 2],         # 5
            [2, -1], [2, 0], [2, 1]]                          # 3

In [51]:
# dilation
def dilation(image, kernel):
    dilation_image  = np.zeros(image.shape, np.uint8)                       
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            max_value = 0
            for k in range(len(kernel)):                                    
                s, t = kernel[k][0] + i, kernel[k][1] + j                   
                if 0 <= s < image.shape[0] and 0 <= t < image.shape[1]:     
                    max_value = max(max_value, image[s, t])                 
            dilation_image[i, j] = max_value                               
    return dilation_image

# erosion
def erosion(image, kernel):
    erosion_image  = np.zeros(image.shape, np.uint8)                       
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            min_value = 255
            for k in range(len(kernel)):                                    
                s, t = kernel[k][0] + i, kernel[k][1] + j                   
                if 0 <= s < image.shape[0] and 0 <= t < image.shape[1]:     
                    min_value = min(min_value, image[s, t])                 
            erosion_image[i, j] = min_value                                 
    return erosion_image

# opening
def opening(image, kernel):
    return dilation(erosion(image, kernel), kernel)

# closing
def closing(image, kernel):
    return erosion(dilation(image, kernel), kernel)

def opening_then_closing(image, kernel):
    return closing(opening(image, kernel), kernel)

def closing_then_opening(image, kernel):
    return opening(closing(image, kernel), kernel)


In [55]:

# aim: generate the opening-then-closing and closing-then-opening filtered images
# subaim: opening-then-closing
opening_then_closing_amp10 = opening_then_closing(Gaussian_10, kernel)
opening_then_closing_amp30 = opening_then_closing(Gaussian_30, kernel)
opening_then_closing_salt_n_pepper_01 = opening_then_closing(salt_n_pepper_01, kernel)
opening_then_closing_salt_n_pepper_005 = opening_then_closing(salt_n_pepper_005, kernel)

# subaim: closing-then-opening
closing_then_opening_amp10 = closing_then_opening(Gaussian_10, kernel)
closing_then_opening_amp30 = closing_then_opening(Gaussian_30, kernel)
closing_then_opening_salt_n_pepper_01 = closing_then_opening(salt_n_pepper_01, kernel)
closing_then_opening_salt_n_pepper_005 = closing_then_opening(salt_n_pepper_005, kernel)

# aim: save the opening-then-closing and closing-then-opening filtered images
# subaim: save the opening-then-closing filtered images
cv2.imwrite('prob_e_result/opening_then_closing_amp10.bmp', opening_then_closing_amp10)
cv2.imwrite('prob_e_result/opening_then_closing_amp30.bmp', opening_then_closing_amp30)
cv2.imwrite('prob_e_result/opening_then_closing_salt_n_pepper_01.bmp', opening_then_closing_salt_n_pepper_01)
cv2.imwrite('prob_e_result/opening_then_closing_salt_n_pepper_005.bmp', opening_then_closing_salt_n_pepper_005)

# subaim: save the closing-then-opening filtered images
cv2.imwrite('prob_e_result/closing_then_opening_amp10.bmp', closing_then_opening_amp10)
cv2.imwrite('prob_e_result/closing_then_opening_amp30.bmp', closing_then_opening_amp30)
cv2.imwrite('prob_e_result/closing_then_opening_salt_n_pepper_01.bmp', closing_then_opening_salt_n_pepper_01)
cv2.imwrite('prob_e_result/closing_then_opening_salt_n_pepper_005.bmp', closing_then_opening_salt_n_pepper_005)

True

# Compute SNR (Signal to Noise Ratio)


In [80]:
def compute_snr(orig_image, filtered_image):

    orig_image = orig_image.astype(float) / 255.0
    filtered_image = filtered_image.astype(float) / 255.0

    m = len(orig_image)
    n = len(orig_image[0])
    mu_orig = 0
    mu_noise = 0
    for i in range(m):
        for j in range(n):
            mu_orig += orig_image[i][j]
            mu_noise += filtered_image[i][j] - orig_image[i][j]
    mu_orig /= m * n
    mu_noise /= m * n
    var_signal = 0
    var_noise = 0
    for i in range(m):
        for j in range(n):
            var_signal += (orig_image[i][j] - mu_orig)**2
            var_noise += (filtered_image[i][j] - orig_image[i][j] - mu_noise)**2
    var_signal /= m * n
    var_noise /= m * n
    return 20 * np.log10(np.sqrt(var_signal) / np.sqrt(var_noise))


In [82]:
print("Gaussian 10 = ", compute_snr(np.array(orig_lena,dtype='int'),Gaussian_10))
print("Gaussian 30 = ", compute_snr(np.array(orig_lena,dtype='int'),Gaussian_30))
print("salt and pepper 0.1 = ", compute_snr(np.array(orig_lena,dtype='int'),salt_n_pepper_01))
print("salt and pepper 0.05 = ", compute_snr(np.array(orig_lena,dtype='int'),salt_n_pepper_005))

print("--------------------------------")
print("3x3 box filter")
print("amplitude 10 = ", compute_snr(np.array(orig_lena,dtype='int'),box_filtered_3x3_amp10))
print("amplitude 30 = ", compute_snr(np.array(orig_lena,dtype='int'),box_filtered_3x3_amp30))
print("salt and pepper 0.1 = ", compute_snr(np.array(orig_lena,dtype='int'),box_filtered_3x3_salt_n_pepper_01))
print("salt and pepper 0.05 = ", compute_snr(np.array(orig_lena,dtype='int'),box_filtered_3x3_salt_n_pepper_005))

print("--------------------------------")
print("5x5 box filter")
print("amplitude 10 = ", compute_snr(np.array(orig_lena,dtype='int'),box_filtered_5x5_amp10))
print("amplitude 30 = ", compute_snr(np.array(orig_lena,dtype='int'),box_filtered_5x5_amp30))
print("salt and pepper 0.1 = ", compute_snr(np.array(orig_lena,dtype='int'),box_filtered_5x5_salt_n_pepper_01))
print("salt and pepper 0.05 = ", compute_snr(np.array(orig_lena,dtype='int'),box_filtered_5x5_salt_n_pepper_005))

print("--------------------------------")
print("3x3 median filter")
print("amplitude 10 = ", compute_snr(np.array(orig_lena,dtype='int'),median_filtered_3x3_amp10))
print("amplitude 30 = ", compute_snr(np.array(orig_lena,dtype='int'),median_filtered_3x3_amp30))
print("salt and pepper 0.1 = ", compute_snr(np.array(orig_lena,dtype='int'),median_filtered_3x3_salt_n_pepper_01))
print("salt and pepper 0.05 = ", compute_snr(np.array(orig_lena,dtype='int'),median_filtered_3x3_salt_n_pepper_005))

print("--------------------------------")
print("5x5 median filter")
print("amplitude 10 = ", compute_snr(np.array(orig_lena,dtype='int'),median_filtered_5x5_amp10))
print("amplitude 30 = ", compute_snr(np.array(orig_lena,dtype='int'),median_filtered_5x5_amp30))
print("salt and pepper 0.1 = ", compute_snr(np.array(orig_lena,dtype='int'),median_filtered_5x5_salt_n_pepper_01))
print("salt and pepper 0.05 = ", compute_snr(np.array(orig_lena,dtype='int'),median_filtered_5x5_salt_n_pepper_005))

print("--------------------------------")
print("opening-then-closing")
print("amplitude 10 = ", compute_snr(np.array(orig_lena,dtype='int'),opening_then_closing_amp10))
print("amplitude 30 = ", compute_snr(np.array(orig_lena,dtype='int'),opening_then_closing_amp30))
print("salt and pepper 0.1 = ", compute_snr(np.array(orig_lena,dtype='int'),opening_then_closing_salt_n_pepper_01))
print("salt and pepper 0.05 = ", compute_snr(np.array(orig_lena,dtype='int'),opening_then_closing_salt_n_pepper_005))

print("--------------------------------")
print("closing-then-opening")
print("amplitude 10 = ", compute_snr(np.array(orig_lena,dtype='int'),closing_then_opening_amp10))
print("amplitude 30 = ", compute_snr(np.array(orig_lena,dtype='int'),closing_then_opening_amp30))
print("salt and pepper 0.1 = ", compute_snr(np.array(orig_lena,dtype='int'),closing_then_opening_salt_n_pepper_01))
print("salt and pepper 0.05 = ", compute_snr(np.array(orig_lena,dtype='int'),closing_then_opening_salt_n_pepper_005))

Gaussian 10 =  13.61167933031741
Gaussian 30 =  4.056464660634249
salt and pepper 0.1 =  -1.8993703672097175
salt and pepper 0.05 =  1.0465169944356754
--------------------------------
3x3 box filter
amplitude 10 =  17.758953655483033
amplitude 30 =  12.530317742185797
salt and pepper 0.1 =  6.538182950530176
salt and pepper 0.05 =  9.566878535232938
--------------------------------
5x5 box filter
amplitude 10 =  14.872069111938153
amplitude 30 =  13.298491072687423
salt and pepper 0.1 =  8.698883639675685
salt and pepper 0.05 =  11.257866464393391
--------------------------------
3x3 median filter
amplitude 10 =  17.6783910321545
amplitude 30 =  11.07863635288467
salt and pepper 0.1 =  15.526709052018514
salt and pepper 0.05 =  19.348865257837666
--------------------------------
5x5 median filter
amplitude 10 =  16.018095018433286
amplitude 30 =  12.90216675003
salt and pepper 0.1 =  15.809263877647005
salt and pepper 0.05 =  16.43482504931115
--------------------------------
opening-